# Aula 07 - Filtragem Baseada em Conhecimento - Exercícios

## Importação dos dados (MovieLens 100k)

In [31]:
import pandas as pd
import numpy as np

In [32]:
import wget
!python3 -m wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!tar -xvzf ml-100k.zip



Saved under ml-100k (2).zip


x ml-100k/
x ml-100k/allbut.pl
x ml-100k/mku.sh
x ml-100k/README
x ml-100k/u.data
x ml-100k/u.genre
x ml-100k/u.info
x ml-100k/u.item
x ml-100k/u.occupation
x ml-100k/u.user
x ml-100k/u1.base
x ml-100k/u1.test
x ml-100k/u2.base
x ml-100k/u2.test
x ml-100k/u3.base
x ml-100k/u3.test
x ml-100k/u4.base
x ml-100k/u4.test
x ml-100k/u5.base
x ml-100k/u5.test
x ml-100k/ua.base
x ml-100k/ua.test
x ml-100k/ub.base
x ml-100k/ub.test


In [33]:
#Types of genres
genre = pd.read_csv('./ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [34]:
#Types of occupations
occupation = pd.read_csv('./ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [35]:
#Load the Ratings data
data = pd.read_csv('./ml-100k/u.data', sep="\t", header=None)
data.columns = ['userId', 'movieId', 'rating', 'timestamp']
data.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [36]:
#Load the Movies data
item = pd.read_csv('./ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movieId', 'title' ,'release','video release date', 'IMDb URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item['release'] = pd.to_datetime(item['release'])
item = item[pd.notnull(item['release'])]
item['year'] = item['release'].dt.year.astype(int)
item.drop(columns=['release', 'video release date', 'IMDb URL'], inplace=True)
item.head()

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995


In [37]:
df_meta = item.melt(id_vars=['movieId', 'title'], var_name='genre')
df_meta = df_meta[df_meta.value == 1]
df_meta.drop(columns=['value'], inplace=True)
# df_meta[df_meta['movieId']==1]
df_meta.head()

,movieId,title,genre
1371,1373,Good Morning (1971),unknown
1682,2,GoldenEye (1995),Action
1684,4,Get Shorty (1995),Action
1697,17,From Dusk Till Dawn (1996),Action
1701,21,Muppet Treasure Island (1996),Action


In [38]:
# Obter a lista de gêneros de um item
def get_genres(df, movieId):
    if movieId not in df['movieId'].values:
        return []
    return df.loc[(df.movieId==movieId),'genre'].tolist()

get_genres(df_meta, 1)

['Animation', "Children's", 'Comedy']

In [39]:
#Load the User data
user = pd.read_csv('./ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['userId', 'age', 'gender', 'occupation', 'zip code']
user.head()

,userId,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


***Exercício 01:*** Implemente um sistema de recomendação baseado em casos que permite as seguintes restrições definidas pelo usuário:
- Escolher entre filmes mais antigos ou mais recentes
- Determinar os gêneors de maior preferência (mas não restrito a eles)
- Escolher os filmes mais populares ou os menos populares dentre os usuários de uma dada profissão.
- Atribuir importância maior ou menor para cada uma das restrições anteriores.

Recomende uma lista de 10 filmes com base nos critérios definidos pelo usuário.

Exemplo: usuário define:
- filmes_mais_recentes = True
- generos_preferidos = ['Drama']
- filmes_mais_populares = True
- profissão = 'writer'
- pesos = [0.33, 0.33, 0.33]

In [41]:
# TODO

# Function to calculate recency similarity (newer or older movies)
def calculate_recency_similarity(movies_df, recent_movies=True):
    # If the user prefers newer movies, newer ones will have higher weight
    max_year = movies_df['year'].max()
    min_year = movies_df['year'].min()
    
    if recent_movies == True:
        movies_df['w_recency'] = (movies_df['year'] - min_year) / (max_year - min_year)
    else:
        movies_df['w_recency'] = (max_year - movies_df['year']) / (max_year - min_year)
    
    return movies_df

# Function to calculate genre similarity (using Jaccard)
def calculate_genre_similarity(movies_df, favorite_genres, df_meta):
    for i, row in movies_df.iterrows():
        # Get the genres of the current movie
        genres = get_genres(df_meta, movies_df.loc[i, 'movieId'])

        # Jaccard similarity
        intersection = len(list(set(genres) & set(favorite_genres)))
        union = len(set(genres + favorite_genres))
        movies_df.loc[i, 'w_genre'] = intersection / union if union > 0 else 0
    
    return movies_df

# Function to calculate movies popularity based on users occupation using the count of ratings
def calculate_movies_popularity_by_occupation(movies_df, data_df, user_df, occupation, most_popular_movies):
    # Filter users by the specified occupation
    profession_users = user_df[user_df['occupation'] == occupation]['userId'].unique()
    
    # Calculate the number of ratings for movies rated by these users
    popularity = data_df[data_df['userId'].isin(profession_users)].groupby('movieId')['rating'].count().reset_index()
    popularity.columns = ['movieId', 'rating_count']
    
    # Merge the popularity data with the movies dataframe
    movies_df = movies_df.merge(popularity, on='movieId', how='left').fillna(0)
    
    # Normalize the popularity and adjust based on the user's preference (most or least popular movies)
    max_count = movies_df['rating_count'].max()
    min_count = movies_df['rating_count'].min()
    
    if most_popular_movies:
        # Normalize: higher count = higher popularity
        movies_df['w_popularity'] = (movies_df['rating_count'] - min_count) / (max_count - min_count)
    else:
        # Inverse normalization: fewer ratings = higher score for less popular movies
        movies_df['w_popularity'] = (max_count - movies_df['rating_count']) / (max_count - min_count)
    
    return movies_df

# Function to calculate the final weighted similarity
def calculate_final_similarity(movies_df, weights):
    movies_df['w'] = weights[0] * movies_df['w_recency'] + weights[1] * movies_df['w_genre'] + weights[2] * movies_df['w_popularity']
    
    return movies_df

# Main function to generate movie recommendations
def get_recommend_movies(movies_df, data_df, df_meta, user_df, recent_movies, favorite_genres, most_popular_movies, occupation, weights):
    # Calculate recency similarity
    movies_df = calculate_recency_similarity(movies_df, recent_movies)
    
    # Calculate genre similarity
    movies_df = calculate_genre_similarity(movies_df, favorite_genres, df_meta)
    
    # Calculate popularity among users of a given occupation
    movies_df = calculate_movies_popularity_by_occupation(movies_df, data_df, user_df, occupation, most_popular_movies)
    
    # Calculate the final similarity based on weights
    movies_df = calculate_final_similarity(movies_df, weights)
    
    # Sort the movies by final similarity and recommend the top 10
    result = movies_df.sort_values(by='w', ascending=False)
    
    return result.head(10)

# Make copies of the dfs, to not change the original ones
movies_df = item.copy()
data_df = data.copy()
user_df = user.copy()
meta_df = df_meta.copy()

get_recommend_movies(movies_df, data_df, meta_df, user_df, True, ['Action', 'Comedy'], True, 'technician', [0.3, 0.3, 0.3])

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Sci-Fi,Thriller,War,Western,year,w_recency,w_genre,rating_count,w_popularity,w
292,294,Liar Liar (1997),0,0,0,0,0,1,0,0,...,0,0,0,0,1997,0.986842,0.500000,16.0,0.727273,0.664234
879,881,Money Talks (1997),0,1,0,0,0,1,0,0,...,0,0,0,0,1997,0.986842,1.000000,3.0,0.136364,0.636962
874,876,Money Talks (1997),0,1,0,0,0,1,0,0,...,0,0,0,0,1997,0.986842,1.000000,2.0,0.090909,0.623325
454,456,Beverly Hills Ninja (1997),0,1,0,0,0,1,0,0,...,0,0,0,0,1997,0.986842,1.000000,2.0,0.090909,0.623325
120,121,Independence Day (ID4) (1996),0,1,0,0,0,0,0,0,...,1,0,1,0,1996,0.973684,0.250000,18.0,0.818182,0.612560
389,391,Last Action Hero (1993),0,1,0,0,0,1,0,0,...,0,0,0,0,1993,0.934211,1.000000,2.0,0.090909,0.607536
180,181,Return of the Jedi (1983),0,1,1,0,0,0,0,0,...,1,0,1,0,1997,0.986842,0.166667,19.0,0.863636,0.605144
386,388,Beverly Hills Cop III (1994),0,1,0,0,0,1,0,0,...,0,0,0,0,1994,0.947368,1.000000,1.0,0.045455,0.597847
1181,1183,"Cowboy Way, The (1994)",0,1,0,0,0,1,0,0,...,0,0,0,0,1994,0.947368,1.000000,1.0,0.045455,0.597847
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,1995,0.960526,0.666667,8.0,0.363636,0.597249


***Exercício 02:*** Considere um CSP definido como uma tripla (V, D, C) onde:
- V = V_usuario U V_filme : variáveis relacionadas com o usuário e o filme, respectivamente
- C = C_usuario U C_filme U C_ui: restrições do usuário, filme e compatibilidade usuário/filme
- D = conjunto de domínios finitos para as variáveis

Podemos definir as variáveis e restrições como:
- V_usuario : {companhia(sozinho, casal, familia), filme_popular(sim, nao), filme_classico(sim, nao)}
- V_filme : {ano_lancamento(int), generos(list), n_avaliacoes(int)}
- C_usuario : {companhia=casal -> filme_popular=sim ^ companhia=sozinho -> filme_classico=nao}
- C_filme : {lista de itens do catalogo}
- C_ui : {companhia=casal -> generos=[Romance, Drama, Comedy], companhia=familia -> generos=[Children's], filme_classico=sim -> ano_lancamento < 1990, filme_classico=nao -> ano_lancamento > 1993, filme_popular=sim -> n_avaliacoes > 100, companhia=sozinho -> generos=[Horror, War]}

Dada a requisição: REQ={companhia=sozinho, filme_popular=sim}, recomende uma lista de 10 filmes para o usuário.

In [74]:
# TODO

# Function to filter movies by popularity
def filter_by_popularity(df_items, df_data, min_ratings):
    # Calculate the average ratings for each movie
    ratings_avg = df_data.groupby('movieId')['rating'].mean().reset_index()
    ratings_count = df_data.groupby('movieId')['rating'].count().reset_index()
    ratings_avg.columns = ['movieId', 'avg_rating']
    ratings_count.columns = ['movieId', 'num_ratings']
    
    # Merge average ratings and number of ratings into the movies dataframe
    df_items = df_items.merge(ratings_avg, on='movieId', how='left').fillna(0)
    df_items = df_items.merge(ratings_count, on='movieId', how='left').fillna(0)
    
    # Filter movies by the minimum number of ratings (popularity constraint)
    df_filtered = df_items[df_items['num_ratings'] > min_ratings]
    
    return df_filtered

# Function to filter movies by classic constraint (year constraint)
def filter_by_classic(movies_df, year_constraint, is_classic):
    if is_classic:
        return movies_df[movies_df['year'] < year_constraint]
    else:
        return movies_df[movies_df['year'] > year_constraint]

# Function to filter movies by genres based on users company preference
def filter_by_genre(movies_df, df_meta, favorite_genres):
    def matches_preferred_genres(movie_id):
        genres = get_genres(df_meta, movie_id)
        return any(genre in favorite_genres for genre in genres)
    
    return movies_df[movies_df['movieId'].apply(matches_preferred_genres)]

# Main function to recommend movies based on CSP constraints
def get_recommend_movies_csp(movies_df, df_meta, df_data, companhia, filme_popular, filme_classico):
    # Define base constraints based on the users company
    if companhia == 'casal':
        favorite_genres = ['Romance', 'Drama', 'Comedy']
    elif companhia == 'familia':
        favorite_genres = ["Children's"]
    elif companhia == 'sozinho':
        favorite_genres = ['Horror', 'War']
    
    # Define movie popularity constraint
    min_ratings = 100 if filme_popular == 'sim' else 0
    
    # Define classic movie constraint (year)
    year_constraint = 1990 if filme_classico == 'sim' else 1993
    
    # Apply filtering
    df_filtered = filter_by_popularity(movies_df, df_data, min_ratings)
    df_filtered = filter_by_classic(df_filtered, year_constraint, filme_classico == 'sim')
    df_filtered = filter_by_genre(df_filtered, df_meta, favorite_genres)
    
    # Sort by average rating and return top 10 movies
    recommended_movies = df_filtered.sort_values(by='avg_rating', ascending=False).head(10)
    
    return recommended_movies


# Make copies of the dfs, to not change the original ones
movies_df = item.copy()
data_df = data.copy()
meta_df = df_meta.copy()

get_recommend_movies_csp(movies_df, meta_df, data_df, companhia='sozinho', filme_popular='sim', filme_classico='nao')


,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,avg_rating,num_ratings
513,515,"Boot, Das (1981)",0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1997,4.203980,201
21,22,Braveheart (1995),0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1996,4.151515,297
180,181,Return of the Jedi (1983),0,1,1,0,0,0,0,0,...,0,0,1,1,0,1,0,1997,4.007890,507
68,69,Forrest Gump (1994),0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,1994,3.853583,321
284,286,"English Patient, The (1996)",0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1996,3.656965,481
30,31,Crimson Tide (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1995,3.629870,154
469,471,Courage Under Fire (1996),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1996,3.610860,221
305,307,"Devil's Advocate, The (1997)",0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1997,3.515957,188
688,690,Seven Years in Tibet (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1997,3.458065,155
286,288,Scream (1996),0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1996,3.441423,478
